In [ ]:
%load_ext autoreload
%autoreload 2

# Mongo

> MongoPool class

In [ ]:
#| default_exp storage.pool.mongo

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Optional, Union, final
import logging
import pandas as pd  # type: ignore
from bson.codec_options import CodecOptions
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.errors import CollectionInvalid

In [ ]:
#| export
from data_io_nbdev.config.db import DBConfig
from data_io_nbdev.data.core import (
    DataFrameDoc,
    ObservationMeta,
    PoolQuery,
    veos_lifetime_start_date,
)
from data_io_nbdev.data.external.pandas_utils import eos_df_to_nested_dict

2023-12-18 18:52:23.169192: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 18:52:23.209462: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 18:52:23.209515: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 18:52:23.209554: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 18:52:23.218923: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 18:52:23.219896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [ ]:
#| export
from data_io_nbdev.storage.pool.pool import Pool  # type: ignore

client_default: MongoClient = MongoClient('mongodb://localhost:27017/')
db_default = client_default['eos']
db_config_default = db_config_servers_by_name["default"]

In [ ]:
#| export
@dataclass(kw_only=True)
class MongoPool(Pool[pd.DataFrame]):
    """
    A MongoDB store for storing data in a time series collection.

    features:

        timeseries interfacing pandas.DataFrame
        multikey index for fast query
        typed hints

    The key leads to a config with db_name and collection name with a switch for record or episode.

    Attributes:

        - meta: ObservationMeta, meta information of the observation
        - db_config: DBConfig, database configuration
        - doc_query: dict, query dict for mongodb
        - codec_option: CodecOptions, codec options for mongodb
        - query: PoolQuery, query for mongodb
        - coll_name: str, collection name
        - collection: Collection, collection for mongodb
        - client: MongoClient, client for mongodb
        - logger: logging.Logger, logger for mongodb
        - dict_logger: dict, dict for logging

    """

    meta: ObservationMeta  # = field(default_factory=ObservationMeta)
    db_config: DBConfig  # db_config_default
    doc_query: dict = field(default_factory=dict)  # query dict for mongodb
    codec_option: CodecOptions
    query: Optional[PoolQuery] = None  # field(default_factory=PoolQuery)
    coll_name: Optional[str] = None  # 'default'
    collection: Optional[Collection] = None  # db_default['default']
    client: Optional[MongoClient] = None  # client_default
    logger: Optional[logging.Logger] = None
    dict_logger: Optional[dict] = None

    def __post_init__(
        self,
    ):
        self.logger = self.logger.getChild("mongo_pool")
        super().__post_init__()
        self.load()

    def load(self):
        mongo_url = "mongodb://" + self.db_config.host + ":" + self.db_config.port
        self.client = MongoClient(
            mongo_url,
            username=self.db_config.user_name,
            password=self.db_config.password,
        )  # mongo_url = 'mongodb://host.docker.internal:27017/'
        version = self.client.server_info()["version"]
        self.logger.info(f"MongoDB version: {version}", extra=self.dict_logger)

        databases = self.client.list_database_names()
        if self.db_config.database_name not in databases:
            self.logger.info(
                f"{self.db_config.database_name} not in {databases}, "
                f"create database {self.db_config.database_name}!",
                extra=self.dict_logger,
            )
        else:
            self.logger.info(f"{self.db_config.database_name} exists.")
        # assert (
        #     self.db_config.database_name in databases
        # ), f"{self.db_config.database_name} not in {databases}!"
        cls = self.__class__

        self.logger.info(f"Concrete type: {cls._type_T}", extra=self.dict_logger)
        # if self.db_config.type == "RECORD":
        #     self.coll_name = self.db_config.collection_name  + self.db_config.type.lower()
        # else:  # type is EPISODE
        #     self.coll_name = self.db_config.collection_name  + self.db_config.type.lower()
        self.coll_name = self.db_config.collection_name

        db = None
        try:
            db = self.client[self.db_config.database_name]
            db.create_collection(
                self.coll_name,
                timeseries={
                    "timeField": "timestamp",  # timestamp as timeField
                    "metaField": "meta",  # plot as meta field
                    "granularity": "seconds",
                },
                codec_options=self.codec_option,  # enable tz aware
                expireAfterSeconds=60 * 60 * 24 * 7 * 365 * 3,  # 3 years
            )
        except CollectionInvalid:
            self.logger.info(f"{self.coll_name} exists.", extra=self.dict_logger)
        except Exception as e:
            self.logger.info(
                f"Cannot get collection {self.coll_name} "
                f"out of database {self.db_config.database_name} "
                f"from {self.db_config.server_name}, "
                f"exception: {e}!",
                extra=self.dict_logger,
            )
            raise e

        # specify new TypedDict for DataFrame

        # if not db:
        #     self.collection: Collection[DataFrameDoc] = db.get_collection(  # type: ignore
        #         self.coll_name
        #     )
        # else:
        self.collection: Collection[DataFrameDoc] = db.get_collection(  # type: ignore
            self.coll_name
        )
        self.doc_query = self.parse_query(self.query)
        self.cnt = self._count(
            self.query
        )  # as a by-product, get the default self.doc_query

    def find_item(self, doc_id: int):
        """
        Find a record by id.
        """
        return self.collection.find_one({"_id": doc_id})

    def close(self):
        self.client.close()
        self.logger.info(f"close mongo client", extra=self.dict_logger)

    @final
    def drop_collection(self):
        """
        Drop all the records in the db.
        """

        self.collection.drop()
        self.logger.info(f"drop collection", extra=self.dict_logger)

    def parse_query(self, query: PoolQuery) -> dict:
        """
        One-Trick Pony: check query is valid or not
        return dict as mongo document query filter
        if query is None, return empty dict
        """

        if query is None:
            return {}

        try:
            epi_start = query.episodestart_start  # .replace(tzinfo=None)
            epi_end = query.episodestart_end  # .replace(tzinfo=None)
        except KeyError as e:
            self.logger.error(f"KeyError: {e}", extra=self.dict_logger)
            raise e

        if self.db_config.type == "RECORD":
            timestamp_start = query.timestamp_start  # .replace(tzinfo=None)
            timestamp_end = query.timestamp_end  # .replace(tzinfo=None)

            if epi_start is None:
                epi_start = (
                    veos_lifetime_start_date.to_pydatetime()
                )  # .replace(tzinfo=None)
            if epi_end is None:
                epi_end = pd.Timestamp.now(
                    tz=self.meta.site.tz,  # "Asia/Shanghai"
                ).to_pydatetime()  # .replace(tzinfo=None)  # until now
            if timestamp_start is None:
                timestamp_start = (
                    veos_lifetime_start_date  # .to_pydatetime().replace(tzinfo=None)
                )
            if timestamp_end is None:
                timestamp_end = pd.Timestamp.now(
                    tz=self.meta.site.tz,  # "Asia/Shanghai"
                ).to_pydatetime()  # .replace(tzinfo=None)  # until now

            doc_query = {
                "$and": [
                    {"meta.vehicle": query.vehicle},
                    {"meta.driver": query.driver},
                    {"meta.episodestart": {"$gt": epi_start}},
                    {"meta.episodestart": {"$lt": epi_end}},
                    {"meta.timestamp": {"$gt": timestamp_start}},
                    {"meta.timestamp": {"$lt": timestamp_end}},
                ]
            }
        else:  # type is EPISODE
            assert (
                type(query) is PoolQuery
            ), "Query type doesn't match db collection type (EPISODE)!"

            if epi_start is None:
                epi_start = (
                    veos_lifetime_start_date.to_pydatetime()
                )  # .replace(tzinfo=None)
            if epi_end is None:
                epi_end = pd.Timestamp.now(
                    tz="Asia/Shanghai"
                ).to_pydatetime()  # .replace(tzinfo=None)  # until now

            if not query.seq_len_from:
                query.seq_len_from = 0

            if not query.seq_len_to:
                query.seq_len_to = int(
                    1e09
                )  # 1 bio steps is enough as upper bound >74k Years

            self.logger.info(
                f"sequence length from {query.seq_len_from} to {query.seq_len_to}",  # type: ignore
                extra=self.dict_logger,
            )
            doc_query = {
                "$and": [
                    {"meta.vehicle": query.vehicle},
                    {"meta.driver": query.driver},
                    {"meta.episodestart": {"$gte": epi_start}},
                    {"meta.episodestart": {"$lte": epi_end}},
                    {"meta.seq_len": {"$gte": query.seq_len_from}},
                    {"meta.seq_len": {"$lte": query.seq_len_to}},
                ]
            }

        return doc_query

    def store_record(self, episode: pd.DataFrame):
        """
        Deposit the records of an episode into the db.
        """

        # encoding DataFrame to nested dict (json format), add meta info then insert_many
        dict_nested = eos_df_to_nested_dict(
            episode
        )  # or 'index'/'tight' nested dict with tuples as keys

        # generate indices info (vehicle, driver, episodestart, timestamp') from DataFrame MultiIndex for meta info
        indices_dict = [
            {episode.index.names[i]: level for i, level in enumerate(levels)}
            for levels in episode.index
        ]
        docs = [
            DataFrameDoc(
                timestamp=idx[
                    "timestamp"
                ]  # redundant, same as in meta['timestamp'] and 'observation'
                .to_pydatetime()
                .replace(
                    microsecond=0  # mongodb timestamp is in BSON Date format, doesn't support microsecond,
                ),  # but only for timestamp, not necessary for timestamps as timestep data
                meta={
                    **idx,
                    **(
                        self.meta.model_dump()
                    ),  # site will dump tz as IANA string as defined in Eoslocation class
                },  # merge two dicts into meta: df.index + ObservationMeta
                observation=dict_nested[key],
            )
            for (idx, key) in zip(indices_dict, dict_nested)
        ]  # list of records, each record is a dict of timestamp, meta, observation (quadruple with timestamp)
        # each row in rows will be a document in MongoDB
        # docs = [{'timestamp': idx["timestamp"].to_pydatetime(),  # redundant, same as in meta['timestamp']
        #         'meta': {**idx, **(self.meta.model_dump())},  # merge two dicts into meta: df.index + ObservationMeta
        #         'observation': dict_nested[key]}
        #         for (idx, key) in zip(indices_dict, dict_nested)]
        # list of records, each record is a dict of timestamp, meta, observation (quadruple with timestamp)

        # use typed collection for type checking
        try:
            result = self.collection.insert_many(docs)
        except Exception as e:
            raise e
        assert result.acknowledged is True, "Record not stored in MongoDB!"
        rec_inserted = self.collection.find({"_id": {"$in": result.inserted_ids}})
        # assert (
        #         set(rec_inserted).symmetric_difference(set(docs))
        #         == set()  # result has to be an empty set if all records are inserted
        # ), "Record stored is not the same as the one inputted!"
        inserted_cnt = len(list(rec_inserted))
        self.cnt = self.cnt + inserted_cnt
        self.logger.info(
            f"'header': 'deposit item number', " f"'inserted item': '{inserted_cnt}'",
            extra=self.dict_logger,
        )
        return result

    def store_episode(self, episode: pd.DataFrame):
        """Deposit a DataFrame of an episode into the db."""

        # convert dataframe episode to dict
        # encoding DataFrame to nested dict (json format), add meta info then insert_many
        dict_nested = eos_df_to_nested_dict(
            episode
        )  # single key of observation timestamp, or 'index'/'tight' nested dict with tuples as keys
        #  convert timestamp key to string for mongodb (only strings are allowed as key for mongodb item key)
        dict_nested = {key.isoformat(): dict_nested[key] for key in dict_nested}

        # generate indices info (vehicle, driver, episodestart, timestamp') from DataFrame MultiIndex for meta info
        indices_dict = [
            {episode.index.names[i]: level for i, level in enumerate(levels)}
            for levels in episode.index
        ]  # all elements in the array should have the same vehicle, driver, episodestart

        meta_episode = indices_dict[0].copy()  # shallow copy is sufficient here
        try:
            meta_episode.pop(
                "timestamp"
            )  # delete the timestamp for the step, the rest are meta info for the episode
        except KeyError:
            self.logger.info(
                f"{{'header': 'timestamp not in the index of the episode!'}}",
                extra=self.dict_logger,
            )
        meta_episode["seq_len"] = len(
            dict_nested
        )  # add sequence length to meta for efficient querying and sampling

        meta = {
            **(self.meta.model_dump()),  # units of measurements
            **meta_episode,  # meta information of the episode, e.g. vehicle, driver, episodestart
        }  # merge two dicts into meta: df.index + ObservationMeta
        doc = DataFrameDoc(
            timestamp=indices_dict[0][
                "episodestart"  # the timestamp of a mongo episode document is the episodestart
            ]  # redundant, same as in meta['timestamp'] and 'observation'
            .to_pydatetime()
            .replace(
                microsecond=0  # mongodb timestamp is in BSON Date format, doesn't support microsecond,
            ),  # but only for timestamp, not necessary for timestamps as timestep data
            meta=meta,  # Done add episodestart to meta
            observation=dict_nested,
        )
        # list of records, each record is a dict of timestamp, meta, observation (quadruple with timestamp)
        # each row in rows will be a document in MongoDB

        # use typed collection for type checking
        result = self.collection.insert_one(doc)
        assert result.acknowledged is True, "Episode not stored in MongoDB!"
        # _ = self.collection.find({"_id": {"$in": result.inserted_id}})
        # assert (
        #         doc_inserted
        #         == doc  # result has to be an empty set if all records are inserted
        # ), "Record stored is not the same as the one inputted!"
        self.cnt = self.cnt + 1
        self.logger.info(f"deposit one item", extra=self.dict_logger)

        return result

    def store(self, episode: pd.DataFrame):
        """
        Deposit the records of an episode into the db.

        Based on the type of the db collection, store the episode as a set of records or as a complete episode.
        """

        if self.db_config.type == "RECORD":
            res = self.store_record(episode)
        else:  # type is EPISODE
            res = self.store_episode(episode)

        return res

    def find(self, query: Union[PoolQuery | PoolQuery]) -> Optional[pd.DataFrame]:
        """
        Find records by a query object.
        """
        doc_query = self.parse_query(query)

        cursor = self.collection.find(doc_query)
        return pd.DataFrame(list(cursor)).drop("_id", axis=1)

    def delete(self, item_id):
        """
        Delete a record by item id.
        """
        self.cnt = self.cnt - 1
        return self.collection.delete_one({"_id": item_id})

    def __iter__(self):
        """
        Iterate over all the records in the db collection
        using generator expression created by Cursor object returned by find().
        """
        return (item for item in self.collection.find())

    def _count(self, query: Optional[PoolQuery] = None):
        """
        Count the number of records in the db.

        for episode/record document
        """

        # if query is not None, doc_query is {}
        doc_query = self.parse_query(query)
        doc_count = self.collection.count_documents(doc_query)

        return doc_count

    def sample(
        self,
        size: int = 4,  # batch size, default 4
        *,
        query: Optional[PoolQuery] = None,  # query for mongodb
    ) -> Optional[pd.DataFrame]:  # samples as one multi-indexed Pandas DataFrame
        """
        Sample a batch of records from the db.

        db.hyperparameters_collection.aggregate([
            {"$match": {"start_time": {"$exists": False}}},
            {"$sample": {"size": 1}}
        ])

        if PoolQuery doesn't contain 'timestamp_start' and 'timestamp_end', the full episode is retrieved.
        """

        assert size > 0

        if query == self.query:  # only if self.query is None can we use self.query
            doc_query = self.doc_query

            if self.cnt <= 0:
                self.logger.info(
                    f"No document for query {query}", extra=self.dict_logger
                )
                return None
            else:
                sample_size = self.cnt
        else:
            self.logger.info(
                f"Non-Default query {query}",
                extra=self.dict_logger,
            )
            doc_query = self.parse_query(query)
            sample_size = self.collection.count_documents(doc_query)

        rest_size = size
        batch: List = []

        self.logger.info(
            f"sample batch from vehicle {query.vehicle} with driver {query.driver}, "
            f"batch_size {size} from document number {sample_size}, "
            f"episodestart from {query.episodestart_start} to {query.episodestart_end}",
            extra=self.dict_logger,
        )
        if self.db_config.type == "RECORD":
            self.logger.info(
                f"timestamp from {query.timestamp_start} to {query.timestamp_end}",  # type: ignore
                extra=self.dict_logger,
            )

        while rest_size > sample_size:
            batch_cursor = self.collection.aggregate(
                [
                    {
                        "$match": doc_query,
                    },
                    {"$sample": {"size": sample_size}},
                ],
                allowDiskUse=True,
            )
            # batch_sampled = list(batch_cursor)
            # sample_size = len(batch_sampled)
            batch = batch + list(batch_cursor)
            rest_size = rest_size - sample_size
        else:  # if
            batch_cursor = self.collection.aggregate(
                [
                    {
                        "$match": doc_query,
                    },
                    {"$sample": {"size": rest_size}},
                ]
            )
            batch = batch + list(batch_cursor)

        return pd.DataFrame(list(batch)).drop("_id", axis=1)

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
show_doc(MongoPool.__post_init__)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L70){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.__post_init__

>      MongoPool.__post_init__ ()

User weakref finalizer to make sure close is called when the object is destroyed

In [ ]:
show_doc(MongoPool.load)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L77){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.load

>      MongoPool.load ()

Initialize the pool interface

This function should:
    - connect to db
    - init

In [ ]:
show_doc(MongoPool.find_item)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L148){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.find_item

>      MongoPool.find_item (doc_id:int)

Find a record by id.

In [ ]:
show_doc(MongoPool.close)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L154){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.close

>      MongoPool.close ()

close the pool, for destructor

In [ ]:
show_doc(MongoPool.parse_query)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L167){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.parse_query

>      MongoPool.parse_query (query:data_io_nbdev.data.core.PoolQuery)

One-Trick Pony: check query is valid or not
return dict as mongo document query filter
if query is None, return empty dict

In [ ]:
show_doc(MongoPool.store_record)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L254){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.store_record

>      MongoPool.store_record (episode:pandas.core.frame.DataFrame)

Deposit the records of an episode into the db.

In [ ]:
show_doc(MongoPool.store_episode)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L314){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.store_episode

>      MongoPool.store_episode (episode:pandas.core.frame.DataFrame)

Deposit a DataFrame of an episode into the db.

In [ ]:
show_doc(MongoPool.store)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L376){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.store

>      MongoPool.store (episode:pandas.core.frame.DataFrame)

Deposit the records of an episode into the db.

Based on the type of the db collection, store the episode as a set of records or as a complete episode.

In [ ]:
show_doc(MongoPool.find)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L388){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.find

>      MongoPool.find (query:data_io_nbdev.data.core.PoolQuery)

Find records by a query object.

In [ ]:
show_doc(MongoPool.delete)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L398){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.delete

>      MongoPool.delete (item_id)

Delete a record by item id.

In [ ]:
show_doc(MongoPool._count)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L412){target="_blank" style="float:right; font-size:smaller"}

### MongoPool._count

>      MongoPool._count (query:Optional[data_io_nbdev.data.core.PoolQuery]=None)

Count the number of records in the db.

for episode/record document

In [ ]:
show_doc(MongoPool.sample)

---

[source](https://github.com/Binjian/data-io-nbdev/tree/main/blob/main/data_io_nbdev/storage/pool/mongo.py#L426){target="_blank" style="float:right; font-size:smaller"}

### MongoPool.sample

>      MongoPool.sample (size:int=4,
>                        query:Optional[data_io_nbdev.data.core.PoolQuery]=None)

Sample a batch of records from the db.

db.hyperparameters_collection.aggregate([
    {"$match": {"start_time": {"$exists": False}}},
    {"$sample": {"size": 1}}
])

if PoolQuery doesn't contain 'timestamp_start' and 'timestamp_end', the full episode is retrieved.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()